In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import os 
from PIL import Image
from tqdm import tqdm
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib.image as img

from fastai.vision.all import *

In [ ]:
TRAINDIR = '../input/petfinder-pawpularity-score/train'
TESTNDIR = '../input/petfinder-pawpularity-score/test'
TrainImgNames = os.listdir(path=TRAINDIR)
TestImgNames = os.listdir(path=TESTNDIR)
TrainDf = pd.read_csv('../input/petfinder-pawpularity-score/'+'train.csv')
TestDf = pd.read_csv('../input/petfinder-pawpularity-score/'+'test.csv')

def Isjpg(fileName):
    if fileName.endswith(".jpg"):
        return True
    else:
        return False

def TestExtenion(ImgNameList):
    count = 0;
    for i in ImgNameList:
        if not Isjpg(i):
            count+=1
    if(count == 0):
        print("All Images")
    
TestExtenion(TrainImgNames)
TestExtenion(TestImgNames)

if (len(TrainDf)==len(TrainImgNames)):
    print(f"Training Images:\t",len(TrainImgNames))
if (len(TestDf)==len(TestImgNames)):
    print(f"Test Images:\t\t",len(TestImgNames))

TrainDf.head()
#Normalising Pawpularity Score
TrainDf['Pawpularity'] = TrainDf['Pawpularity']/100

In [ ]:
def PickRandomImage():
    i = np.random.randint(len(TrainImgNames))
    imgName = TrainImgNames[i]
    im = img.imread(os.path.join(TRAINDIR,imgName))
    return im,imgName

def Pawpularity(x):
    if x.endswith('.jpg'):
        id = x[:-4]
        x = TrainDf[TrainDf['Id'] == id]
        return float(x['Pawpularity'])

(_,name) = PickRandomImage()
print(Pawpularity(name))

In [ ]:
dls = ImageDataLoaders.from_name_func(TRAINDIR,get_image_files(TRAINDIR),label_func=Pawpularity,valid_pct=0.2,item_tfms=Resize(224))

In [ ]:
# Model 2
learn = cnn_learner(dls, resnet152, metrics=accuracy)
learn.fine_tune(15)
learn.save("Model2",'./')

In [ ]:
#Model 1
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(1)
learn.save("Model1",'./')
#overfit at epoch 2